In [2]:
import pickle
import torch
import numpy as np
import MF
import math
from sklearn import cluster
from operator import itemgetter
from math import log
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import argparse
import pandas as pd
import utility
from scipy.sparse import csr_matrix, rand as sprand
from tqdm import tqdm
from MF import MF
import VAE
import os


np.random.seed(0)
torch.manual_seed(0)

In [7]:
### Unpickling user and item matrices

### mtx_item[I]: the list of users that like item I
### mtx_user[U]: the list of items that user U like

with open("./data/item", "rb") as f: 
    mtx_item = pickle.load(f)

with open("./data/user", "rb") as f:
    mtx_user = pickle.load(f)
    
### Read the trained model
model = torch.load("./data/save.pt")
# Item matrix
Q = model.item_factors.weight.H.tolist()
# User matrix
P = model.user_factors.weight.H.tolist()

P = np.array(P)
Q = np.array(Q)

### Predicted matrix
Rec = np.matmul(P.T, Q)


NameError: name 'shape' is not defined

In [ ]:
### Functions for metrices
def get_activeness(user):
    return len(mtx_user[user])

def get_popularity(item):
    return len(mtx_item[item])

def get_user_mainstreamness(user):
    ### mainstreamness == jacarrd similarity
    j_sim = 0
    for t in range(len(mtx_user)):
        if t == user:
            continue
        dul_list = [x for x in mtx_user[user] if x in mtx_user[t]]
        l = len(dul_list)
        j_sim += l/(len(mtx_user[user]+mtx_user[t])-l)
    j_sim /= len(mtx_user)-1
    return(j_sim)

def get_item_mainstreamness(item):
    ### mainstreamness == jacarrd similarity
    j_sim = 0
    for t in range(len(mtx_item)):
        if t == item:
            continue
        dul_list = [x for x in mtx_item[item] if x in mtx_item[t]]
        l = len(dul_list)
        num_sim = (len(mtx_item[item]+mtx_item[t])-l)
        if num_sim == 0:
            j_sim += 1
        else:
            j_sim += l/num_sim
    j_sim /= len(mtx_item)-1
    return(j_sim)



In [ ]:
### Calculate the metrices

activeness = []
user_mainstreamness = []
for t in range(6040):
    activeness.append(get_activeness(t))
    user_mainstreamness.append((get_user_mainstreamness(t)))
                               
popularity = []
item_mainstreamness = []
for t in range(3706):
    popularity.append(get_popularity(t))
    item_mainstreamness.append((get_item_mainstreamness(t)))

In [ ]:
act_sort = np.sort(np.array(activeness))
usrmain_sort = np.sort(np.array(user_mainstreamness))
pop_sort = np.sort(np.array(popularity))
itmmain_sort = np.sort(np.array(item_mainstreamness))
print(act_sort)
print(usrmain_sort)
print(pop_sort)
print(itmmain_sort)

In [ ]:
np.save("./data/activeness.npy", activeness)
np.save("./data/user_mainstreamness.npy", user_mainstreamness)
np.save("./data/popularity.npy", popularity)
np.save("./data/item_mainstreamness.npy", item_mainstreamness)

np.save("./data/act_sort.npy", act_sort)
np.save("./data/usrmain_sort.npy", usrmain_sort)
np.save("./data/pop_sort.npy", pop_sort)
np.save("./data/itmmain_sort.npy", itmmain_sort)

In [9]:
### Load the metrics from disk
act_sort = np.load("./data/act_sort.npy")
pop_sort = np.load("./data/pop_sort.npy")
usrmain_sort = np.load("./data/usrmain_sort.npy")
itmmain_sort = np.load("./data/itmmain_sort.npy")

In [11]:
test_like = np.load("./data/user_vali_like.npy", allow_pickle=True)
train_like = np.load("./data/user_train_like.npy", allow_pickle=True)

In [12]:
### The test_like and train_like lists, but for item
### test_like_item[I]: the users who like item I in the test set
### train_like_item[I]: the users who like item I in the train set
test_like_item = []
for t in range(Rec.shape[1]):
    test_like_item.append([])
i = 0
for t in test_like:
    for tt in t:
        test_like_item[tt].append(i)
    i += 1

train_like_item = []
for t in range(Rec.shape[0]):
    train_like_item.append([])
i = 0
for t in train_like:
    for tt in t:
        train_like_item[tt].append(i)
    i += 1

In [ ]:
### Functions for calculating NDCG@K and performance

num_u = Rec.shape[1]
like = train_like
for i in range(num_u):
    Rec[i, like[i]] = -100000.0
    
Rec_sort = []
for t in Rec:
    num = t.tolist().count(-100000.0)
    tt = np.argsort(t)[::-1]
    tt = tt[0:(len(tt)-num)]
    Rec_sort.append(tt)

def NDCG_at_k(predicted_list, ground_truth, k):
    dcg_value = [(v / log(i + 1 + 1, 2)) for i, v in enumerate(predicted_list[:k])]
    dcg = np.sum(dcg_value)
    if len(ground_truth) < k:
        ground_truth += [0 for i in range(k - len(ground_truth))]
    idcg_value = [(v / log(i + 1 + 1, 2)) for i, v in enumerate(ground_truth[:k])]
    idcg = np.sum(idcg_value)
    return dcg / idcg

def get_ndcg(user):
    k_set = [1,5,10,20]
    u_pred = Rec[u, :]
    top15_item_idx_no_train = np.argpartition(u_pred, -k_set[-1])[-k_set[-1]:]
    top15 = (np.array([top15_item_idx_no_train, u_pred[top15_item_idx_no_train]])).T
    top15 = sorted(top15, key=itemgetter(1), reverse=True)
    new_user_prediction = top15
    test = test_like[user]

    dcg_list = []

    # compute the number of true positive items at top k
    count_1, count_5, count_10, count_15 = 0, 0, 0, 0
    for i in range(k_set[3]):
        if i < k_set[0] and new_user_prediction[i][0] in test:
            count_1 += 1.0
        if i < k_set[1] and new_user_prediction[i][0] in test:
            count_5 += 1.0
        if i < k_set[2] and new_user_prediction[i][0] in test:
            count_10 += 1.0
        if new_user_prediction[i][0] in test:
            count_15 += 1.0
            dcg_list.append(1)
        else:
            dcg_list.append(0)

    # calculate NDCG@k
    idcg_list = [1 for i in range(len(test))]
    ndcg_tmp_1 = NDCG_at_k(dcg_list, idcg_list, k_set[0])
    ndcg_tmp_5 = NDCG_at_k(dcg_list, idcg_list, k_set[1])
    ndcg_tmp_10 = NDCG_at_k(dcg_list, idcg_list, k_set[2])
    ndcg_tmp_15 = NDCG_at_k(dcg_list, idcg_list, k_set[3])

    return ndcg_tmp_15



def get_performance(item):
    test = test_like_item[item]
    ### Number of the users who likes this item
    hit_usr = len(test)
    ### Top of the fraction
    a = 0
    for t in test:
        ### Column here
        ranklist = Rec_sort[t]
        rank = ranklist.tolist().index(item)+1
        a += 1/math.log(rank+2, 2)
    if hit_usr != 0:
        performance = a/hit_usr
    else:
        performance = 0
    return performance

In [ ]:
def evaluate_user(mtc, k):
    ### mtc should be index of the users
    ### K for NDCG@K
    k_ndcg = 15
    
    ### Divide users into k groups evenly
    bt = int(len(mtc)/k)
    labels = []
    for t in range(k):
        if t == k-1:
            labels[t*bt::] = [t]*(len(mtc)-(k-1)*bt)
        else:
            labels[t*bt:t*(bt+1)] = [t]*bt
        
    NDCG = {}
    ave_mtc = {}
    m = {}
    for t in range(k):
        NDCG[t] = []
        ave_mtc[t] = 0
        n = 0
        m[t] = []
        for ii in range(len(labels)):
            if labels[ii] == t:
                NDCG[t].append(get_ndcg(ii))
                ave_mtc[t] += mtc[ii]
                n += 1
                m[t].append(mtc[ii])
        ave_mtc[t] /= n
    return NDCG, ave_mtc, m

def evaluate_item(mtc, k):
    clusters = cluster.KMeans(n_clusters = k).fit(mtc.reshape(-1, 1))
    performance = {}
    ave_mtc = {}
    m = {}
    for t in range(k):
        performance[t] = []
        ave_mtc[t] = 0
        n = 0
        m[t] = []
        for ii in range(len(clusters.labels_)):
            if clusters.labels_[ii] == t:
                performance[t].append(get_performance(ii))
                ave_mtc[t] += mtc[ii]
                n += 1
                m[t].append(mtc[ii])
        ave_mtc[t] /= n
    return performance, ave_mtc, m


In [ ]:
res_usrmain, ave_usrmain, m_usrmain = evaluate_user(usrmain_sort, 10)
res_act, ave_act, m_act = evaluate_user(act_sort, 10)
res_itmmain, ave_itmmain, m_itmmain = evaluate_item(itmmain_sort, 10)
res_pop, ave_pop, m_pop = evaluate_item(pop_sort, 10)

In [ ]:
### Convert results to dataframe
def to_df(r, a, m):
    df = {"result": [], "metric": [], "group": [], "ave_metric": []}
    k = len(r)
    for t in range(k):
        l = len(r[t])
        for tt in range(l):
            df["result"].append(r[t][tt])
            df["metric"].append(m[t][tt])
            df["group"].append(t)
            df["ave_metric"].append(a[t])
    df = pd.DataFrame(df)
    return df

In [ ]:
### Save the dataframes
df = to_df(res_pop, ave_pop, m_pop)
df.to_csv("./data/item pop.csv")
df = to_df(res_usrmain, ave_usrmain, m_usrmain)
df.to_csv("./data/user main.csv")
df = to_df(res_itmmain, ave_itmmain, m_itmmain)
df.to_csv("./data/item main.csv")
df = to_df(res_act, ave_act, m_act)
df.to_csv("./data/user act.csv")
